In [194]:
from sqlalchemy import (
    Column, Integer, String, Text, ForeignKey, DateTime, Boolean, Enum, Index, Table, UniqueConstraint, JSON, Float, event, inspect
)
from sqlalchemy.sql import func
from mixins import TimestampMixin
from sqlalchemy.orm import relationship, declarative_base, Session
from sqlalchemy import create_engine

from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
from datetime import datetime
from sqlalchemy.dialects.postgresql import JSONB
from dotenv import load_dotenv
import os
import boto3
import enum

ModuleNotFoundError: No module named 'mixins'

# Step 0. Imports & Base

In [2]:
# from sqlalchemy import create_engine, text

# # load the .env file
# load_dotenv()

# # get database url
# DATABASE_URL = os.getenv("DATABASE_URL")

# engine = create_engine(DATABASE_URL, echo=True)

# with engine.connect() as conn:
#     result = conn.execute(text("SELECT version();"))
#     print("✅ Connected to:", result.scalar())

2025-09-21 14:00:55,055 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-09-21 14:00:55,055 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 14:00:55,056 INFO sqlalchemy.engine.Engine select current_schema()
2025-09-21 14:00:55,056 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 14:00:55,057 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-09-21 14:00:55,057 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 14:00:55,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 14:00:55,058 INFO sqlalchemy.engine.Engine SELECT version();
2025-09-21 14:00:55,058 INFO sqlalchemy.engine.Engine [generated in 0.00052s] {}
✅ Connected to: PostgreSQL 14.18 (Homebrew) on aarch64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit
2025-09-21 14:00:55,059 INFO sqlalchemy.engine.Engine ROLLBACK


### CAUTION: SQLAlchemy’s in-memory registry.
When you re-run your model definitions (class Project(Base) etc.), SQLAlchemy thinks you’re trying to define the same table again in the same Python session.

Fixes:

* Restart the kernel (clean slate).

* Or run Base.metadata.clear() before re-defining models.

* Or (not recommended for production, but useful in notebooks) add:    

`__table_args__ = {"extend_existing": True}`    

inside each model.    

**Potential problems of 3rd option-- Silent overwrites**    

If you redefine a model with different column definitions, SQLAlchemy will happily overwrite the in-memory Python mapping.    

But the database table itself is unchanged — unless you drop/recreate or run a migration.    

This can cause a mismatch: your Python code thinks a column exists (or has a new type), but the real Postgres table does not.    

### My own suggestion:    
#### During prototyping stage:         
* use `__table_args__ = {"extend_existing": True}` for each model;
    
#### When your schema stabilizes and you’re preparing for AWS deployment:    
1. Move your models into models.py (or a models/ package).

        Delete __table_args__ = {"extend_existing": True} from each model.
        Define Base = declarative_base() once at the top.

2. Add Alembic to manage schema evolution:

    `pip install alembic`    

    `alembic init migrations`    

* Configure alembic.ini with your DATABASE_URL.   

* In env.py, set target_metadata = Base.metadata.

3. Whenever you change a model:

    `alembic revision --autogenerate -m "describe change"`
   
    `alembic upgrade head`

This will safely apply only the changes needed, without dropping your tables.

In [ ]:
# create base class
Base = declarative_base() 

#---
# It creates a registry (Base.metadata) that will hold all the tables you define.
# Every time you define a model (class Project(Base): ...), that model’s table gets registered into Base.metadata.tables.
#---

In [ ]:
# Base.metadata.clear()

# Step 1. Enums

In [ ]:
# --------------------------
# Enums
# --------------------------

## 1.1 ProjectStatus
class ProjectStatus(enum.Enum):
    draft = "draft"                  # project created, requirements being defined
    ready_for_annotation = "ready_for_annotation"  # files uploaded, jobs not started
    in_progress = "in_progress"      # annotation jobs are running
    completed = "completed"          # all jobs done
    archived = "archived"            # project closed, read-only

## 1.2 FileStatus (file lifecycle)
class FileStatus(enum.Enum):
    pending = "pending"
    ready_for_annotation = "ready_for_annotation"
    in_progress = "in_progress"
    completed = "completed"
    archived = "archived"

## 1.3 FileType
class FileType(enum.Enum):
    dataset = "dataset"
    requirement = "requirement"
    report = "annotation_results"
    llm_output = "llm_output"
# Does our PM also needs to upload sliced file results? (NO currently)

# ## 1.4 UserRole
# class UserRole(enum.Enum):
#     org_admin = "org_admin"       # customer admin
#     org_pm = "org_pm"             # customer project manager
#     our_pm = "our_pm"             # our company PM that manages annotation jobs & assigns annotators
#     annotator = "annotator"       # our company annotator
#     qc = "qc"                     # our company QC for annotation results review

## 1.5 AnnotationJobStatus (job lifecycle)
class AnnotationJobStatus(enum.Enum):
    not_started = "not_started"
    in_progress = "in_progress"
    submitted = "submitted"
    reviewed = "reviewed"

## 1.6 ReviewStatus
class ReviewStatus(enum.Enum):
    pending = "pending"
    approved = "approved"
    rejected = "rejected"

## 1.7 EntityType
class EntityType(enum.Enum):
    project = "project"
    file = "file"
    file_version = "file_version"
    annotation_job = "annotation_job"

## 1.8 EventType
class EventType(enum.Enum):
    uploaded = "uploaded"
    reuploaded = "reuploaded"
    annotation_started = "annotation_started"
    annotation_completed = "annotation_completed"
    reviewed = "reviewed"
    deleted = "deleted"
    status_changed = "status_changed"

## 1.9 AssignmentRole
class AssignmentRole(enum.Enum):
    annotator = "annotator"
    reviewer = "reviewer"
    qc = "qc"   # quality control / audit

## 1.10 Language (for AnnotationJob)
class Language(enum.Enum):
    en = "en"   # English
    zh = "zh"   # Chinese
    fr = "fr"   # French
    de = "de"   # German
    es = "es"   # Spanish
    ar = "ar"   # Arabic

## 1.11 Priority (for AnnotationJob)
class JobPriority(enum.Enum):
    low = "low"
    medium = "medium"
    high = "high"

# Step 2. Association Tables 

In [ ]:
# --------------------------
# Association Tables
# --------------------------

# 2.1 User <-> Role
user_roles = Table(
    "user_roles",
    Base.metadata,
    Column("user_id", Integer, ForeignKey("user.user_id", ondelete="CASCADE"), primary_key=True),
    Column("role_id", Integer, ForeignKey("role.role_id", ondelete="CASCADE"), primary_key=True),
    extend_existing=True  
)

# 2.2 Role <-> Permission
role_permissions = Table(
    "role_permissions",
    Base.metadata,
    Column("role_id", Integer, ForeignKey("role.role_id", ondelete="CASCADE"), primary_key=True),
    Column("permission_id", Integer, ForeignKey("permission.permission_id", ondelete="CASCADE"), primary_key=True),
    extend_existing=True  
)

# This table records which annotators have worked on this job before 
# (for feedback loops / reassignment tracking)
job_previous_annotators = Table(
    "job_previous_annotators",
    Base.metadata,
    Column("job_id", Integer, ForeignKey("annotation_job.job_id", ondelete="CASCADE"), primary_key=True),
    Column("user_id", Integer, ForeignKey("user.user_id", ondelete="CASCADE"), primary_key=True),
    Column("assigned_at", DateTime, default=func.now()),
    extend_existing=True  
)


# One ExportLog may include multiple versions.
# One FileVersion may appear in multiple exports
class ExportedFile(Base):
    __tablename__ = "exported_file"
    __table_args__ = {"extend_existing": True}

    export_id = Column(Integer, ForeignKey("export_log.export_id", ondelete="CASCADE"), primary_key=True)
    file_version_id = Column(Integer, ForeignKey("file_version.version_id", ondelete="CASCADE"), primary_key=True)
    included_at = Column(DateTime, default=func.now())

    # Relationships
    export = relationship("ExportLog", back_populates="exported_files")
    file_version = relationship("FileVersion", back_populates="exported_files")



# Step 3: CORE TABLES

In [ ]:
REAL_UPDATE_COLS = {
    # Project life-cycle
    "project": {
        "status",         # workflow stage
        "name",           # user-visible name
        "description",    # requirement details
        "client_pm_id",   # who owns it on client side
        "our_pm_id",      # our internal PM
        "is_active"       # soft enable/disable
    },

    # Files inside a project
    "file": {
        "status",         # processing state
        "name",
        "description",
        "active_version_id",  # which version is served
        "is_active"
    },

    # Individual file versions (mostly immutable)
    "file_version": {
        "is_active",          # whether this version is live
        "generation_method",  # upload / OCR / LLM
        "llm_model",          # model used
        "llm_params"          # model parameters
    },

    # FileTable for the file uploaded by the client company
    "file_table": {
    "status",
    "description",
    "schema_json",
    "extracted_narrative",
    "is_active"
},

    # Human users
    "user": {
        "email",              # login identifier
        "org_id",             # organization assignment
        "availability",       # capacity
        "language_expertise", # skill profile
        "skill_score",        # QA/score metrics
        "skill_level",
        "qa_approval_rate",   # performance metrics
        "is_active"
    },

    # Annotation jobs
    "annotation_job": {
        "status",        # job workflow
        "review_status", # review result
        "priority",
        "language",
        "due_date",
        "is_active"
    },

    # Assignment of a job to a user
    "assignment": {
        "status",  # assigned → in_progress → completed
        "role",
        "user_id"
    },

    # Reviews on completed jobs
    "review": {
        "status",   # pending / approved / rejected
        "feedback", # reviewer comments
        "is_active"
    },

    # Export operations
    "export_log": {
        "status",       # pending / completed / failed
        "storage_path", # destination location
        "checksum"
    },

    # Organization meta
    "organization": {
        "name",
        "description",
        "is_active"
    },

    # Event log rows are append-only — no updates expected.
    # If you ever allow edits, list those columns here.
    # "event_log": set(),
}

## 3.1 Project Table

In [170]:
# -----------------------------
# Core Tables
# -----------------------------
    
# Project Table
class Project(Base, TimestampMixin):
    __tablename__ = "project"
    __table_args__ = (
    UniqueConstraint("org_id", "name", name="uq_org_project_name"), # no two projects can share same name in one comp
    Index("ix_project_status", "status"), # speeds up dashboards like “show me all in-progress projects”.
    Index("ix_project_is_active", "is_active"), # speeds up “only show active projects”.
    Index("ix_project_client_pm_id", "client_pm_id"), # useful if query “all projects started by this PM”.
    Index("ix_project_org_id", "org_id"), # useful if query “all projects for this org”.
    {"extend_existing": True} # delete
    )

    project_id = Column(Integer, primary_key=True, autoincrement=True)
    org_id = Column(Integer, ForeignKey("organization.org_id"), nullable=False)
    name = Column(String, nullable=False)
    description = Column(Text, nullable=True) # longer desp than name

    # plain text instructions
    requirements_text = Column(Text, nullable=True)
    # optional uploaded doc (PDF, Word, PPT, etc.)
    # requirements_file_id = Column(Integer, ForeignKey("file.file_id"), nullable=True)

    # project status enum
    status = Column(Enum(ProjectStatus, name="project_status_enum"), default=ProjectStatus.draft)
    
    is_active = Column(Boolean, default=True, nullable=False)

    date_created = Column(DateTime, default=func.now(), nullable=False)
    date_updated = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)
    completed_at = Column(DateTime, nullable=True)
    deleted_at = Column(DateTime, nullable=True) # when customer delete the project

    # --- PM links ---
    client_pm_id = Column(Integer, ForeignKey("user.user_id"), nullable=False) # client PM
    our_pm_id = Column(Integer, ForeignKey("user.user_id"), nullable=True) # our PM

    # --- Relationships ---
    files = relationship("File", back_populates="project")             # all files
    # convenience: only requirement files
    requirement_files = relationship(
        "File",
        primaryjoin="and_(Project.project_id==File.project_id, File.file_type=='requirement')",
        viewonly=True
    ) # only get files that accords with reqs
    jobs = relationship("AnnotationJob", back_populates="project")     # all jobs
    events = relationship("EventLog", back_populates="project")        # all events
    organization = relationship("Organization", back_populates="projects")
    client_pm = relationship("User", foreign_keys=[client_pm_id], back_populates="client_projects")
    our_pm = relationship("User", foreign_keys=[our_pm_id], back_populates="managed_projects")
    exports = relationship("ExportLog", back_populates="project")


/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/3851405211.py:6: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Project, and will be replaced in the string-lookup table.
  class Project(Base, TimestampMixin):


## 3.2 File Table 
(currently only create one for all kinds of files' storage)

check the logic:
1. files that clients uploaded, our PM needs to be able to view it so that they can further assign tasks;     
2. files that clients uploaded, needs to be able to transport to LLM model(maybe it's sth in backend?)     
3. annotator also needs relevant access, because for each task they needs the correspond raw file for annotation   
4. quality checker also need them to check the work done by the annotator.     
5. clients needs to be able to view the files they had uploaded and see the progress     
6. organizations also needs to be able to view all the projects as well as all the files their company uploaded and created.    
7. for the llm generated file, the annotator need to have access to it for corresponding task;     
8. for annotated file uploaded by annotators, they needs to be able to send to quality checker;     
9. the files that's approved by quality check needs to be able to back to customers     
10. the organization needs to have access to all the final annotated files     
11. the client PM needs to have access to the final annotated files that they uploaded     
12. during uploading, each project allow customer to upload multiple files with different versions, only by clicking sth like "confirmation" will one project be created, and when project created, all files will be in pending status 
13. raw files can be deleted by customers before the task is processing (can be deleted during pending status)     
14. one project only related to one client, but one client can relate to many projects     
15. one file only related to one projects, but only projects can have multiple files,     
16. when deleting a project, all the files under this project will be inactive,    
17. both project manager in client company and the corresponding organization can have the access to delete the project(but client can only delete the project they created)    
18. further link with file size and format

In [171]:
# --------------------------
# File Table
# --------------------------
class File(Base, TimestampMixin):
    __tablename__ = "file"
    __table_args__ = (
    UniqueConstraint("project_id", "name", name="uq_project_file_name"),
    Index("ix_file_project_id", "project_id"), # speeds up “all files in project.”
    Index("ix_file_status", "status"), # speeds up “all files ready for annotation.”
    Index("ix_file_type", "file_type"), # speeds up filtering datasets vs. requirements.
    {"extend_existing": True},
)


    file_id = Column(Integer, primary_key=True, autoincrement=True)
    # descriptive file name (user-facing)
    name = Column(String, nullable=False)
    description = Column(Text, nullable=True)
    uploaded_by = Column(Integer, ForeignKey("user.user_id"), nullable=False)


    # workflow state
    status = Column(
        Enum(FileStatus, name="file_status_enum"),
        default=FileStatus.pending,
        nullable=False
    )
    # what kind of file this is (dataset, requirement, annotation_results, llm_nl)
    file_type = Column(Enum(FileType, name="file_type_enum"), nullable=False, default=FileType.dataset)

    # audit timestamps
    date_created = Column(DateTime, default=func.now(), nullable=False)
    date_updated = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)
    is_active = Column(Boolean, default=True, nullable=False)
    deleted_at = Column(DateTime, nullable=True)

    # validate uploads by storage
    size_bytes = Column(Integer, nullable=True)
    mime_type = Column(String, nullable=True) # technical format

    
    # --- PM links ---
    project_id = Column(Integer, ForeignKey("project.project_id"), nullable=False)
    # active version pointer
    active_version_id = Column(Integer, ForeignKey("file_version.version_id"), nullable=True)


    # --- Relationships ---
    uploader = relationship("User", back_populates="uploaded_files")
    project = relationship("Project", back_populates="files")
    versions = relationship("FileVersion", back_populates="file", cascade="all, delete-orphan")
    events = relationship("EventLog", back_populates="file")
    active_version = relationship("FileVersion", foreign_keys=[active_version_id], uselist=False)
    tables = relationship("FileTable", back_populates="file", cascade="all, delete-orphan")

/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/87485939.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.File, and will be replaced in the string-lookup table.
  class File(Base, TimestampMixin):


## 3.3 File Version Table

In [172]:
# --------------------------
# Version Table
# --------------------------
class FileVersion(Base, TimestampMixin):
    __tablename__ = "file_version"
    __table_args__ = (
        Index("ix_fileversion_file_id", "file_id"),
        {"extend_existing": True},
    )

    version_id = Column(Integer, primary_key=True, autoincrement=True)

    # --- Parent link ---
    file_id = Column(Integer, ForeignKey("file.file_id", ondelete="CASCADE"), nullable=False)
    version_number = Column(Integer, nullable=False)  # 1, 2, 3…

    # --- Storage info ---
    storage_path = Column(String, nullable=False)   # MinIO/S3 key or path
    checksum = Column(String, nullable=True)        # for integrity validation
    size_bytes = Column(Integer, nullable=True)     # optional: store size at version-level
    mime_type = Column(String, nullable=True)       # optional: file format at version-level

    # --- Upload & provenance ---
    uploaded_by = Column(Integer, ForeignKey("user.user_id"), nullable=True)
    uploaded_at = Column(DateTime(timezone=True), server_default=func.now(), nullable=False)
    # --- Lifecycle flags ---
    is_active = Column(Boolean, default=True, nullable=False)

    source_file_version_id = Column(Integer, ForeignKey("file_version.version_id"), nullable=True)
    generation_method = Column(
        Enum("upload", "ocr", "llm", name="generation_method_enum"),
        default="upload",
        nullable=False
    )
    llm_model = Column(String, nullable=True)       # e.g., "gpt-4", "llama-3"
    llm_params = Column(JSON, nullable=True)        # parameters if generated by LLM


    # --- Relationships ---
    file = relationship("File", back_populates="versions")
    source_version = relationship("FileVersion", remote_side=[version_id])  # self-ref
    events = relationship("EventLog", back_populates="file_version")        # version-level logs
    exports = relationship(
    "ExportLog",
    secondary="exported_file",
    back_populates="file_versions"
    )


    # export_id = Column(Integer, ForeignKey("export_log.export_id"), nullable=True)
    exported_files = relationship(
        "ExportedFile",
        back_populates="file_version",
        cascade="all, delete-orphan"
    )


/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/1825405250.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.FileVersion, and will be replaced in the string-lookup table.
  class FileVersion(Base, TimestampMixin):


## 3.4 FileTable Table

the logic of this table should start with, the project manager in other company, they create a project, and in the project, there may upload multiple files for annotation, that's ok, but then these files will be put in LLM and LLM will partition them into tables, and generate natural language for that file; and our product manager will assign each table and the corresponding natural language generated by llm to different annotators, and each annotator will do further annotation, also later when each task is finished, they will be given to quality checker for checking and they may choose to complete this project or send back with feedback to annotators to modify their annotations, for each table is linked with a file, and each file can have multiple tables, when a file is pending, product managers from other company can re-upload this file, if all the files are pending, they can even delete the project; but if they are in process, they are no longer allowed to delete, and finally if all the tables are completed by quality checker, then in product manager's interfaceI(both our company and client company) it shows that the file is finished; if all the files are completed, then in product manager's interface(both our company pm and the client company pm) it shows that the project is completed and PM in client's company are allowed to download everything as a zip. so at first, the progress is tracked by file, then the task is linked to each table and all the tables status lead to the status of the file and all the files' status lead to the project

In [ ]:
class FileTable(Base, TimestampMixin):
    __tablename__ = "file_table"
    __table_args__ = (
        Index("ix_table_file_id", "file_id"),
        Index("ix_table_status", "status"),
        {"extend_existing": True},
    )

    table_id = Column(Integer, primary_key=True, autoincrement=True)

    # Each table belongs to exactly one file
    file_id = Column(Integer, ForeignKey("file.file_id", ondelete="CASCADE"), nullable=False)
    version_id = Column(Integer, ForeignKey("file_version.version_id", ondelete="SET NULL"), nullable=True)

    # Metadata for traceability
    name = Column(String, nullable=True)      # optional, e.g., "Table 3: Revenue Summary"
    description = Column(Text, nullable=True) # optional text or auto-detected caption

    # JSON column to store table schema or preview content
    schema_json = Column(JSONB, nullable=True)  # {"columns": ["col1", "col2"], "types": ["str", "float"]}
    extracted_narrative = Column(Text, nullable=True)  # the LLM-generated NL paragraph for that table

    # Table-level workflow
    status = Column(
        Enum(FileStatus, name="filetable_status_enum"),
        default=FileStatus.pending,
        nullable=False
    )

    # Audit
    is_active = Column(Boolean, default=True, nullable=False)
    deleted_at = Column(DateTime, nullable=True)

    # --- Relationships ---
    file = relationship("File", back_populates="tables")
    version = relationship("FileVersion", back_populates="tables", foreign_keys=[version_id])
    jobs = relationship("AnnotationJob", back_populates="table", cascade="all, delete-orphan")
    events = relationship("EventLog", back_populates="table", cascade="all, delete-orphan")
    #tables = relationship("FileTable", back_populates="version")

## 3.4 User Table

"user" is a generic account table that represents any actor in the system:

* Organization Admin (client company, oversees all projects).

* Organization PM (client company, uploads datasets + requirements).

* Our PM (your company, manages annotation jobs & assigns annotators).

* Annotators (our company, upload results).

* QC / Reviewers (our company, upload corrections).

therefore, "user" here basically means everyone who logs in and interacts with the system.

product manager(our company):

1. able to check the availability of the annotators and quality checkers
2. able to check for each file, who and how many annotators have annotated that
3. able to check the files uploaded by the client company’s PM or uploaded by the organization
4. have access to projects and further assign annotators to each project to conduct annotation job 
5. have access to reassign annotators for same project
6. have access to assign quality checker (one people can both be quality checker and annotator the same time, but for each project, if one person can either be quality checker or annotator)

client company’s PM:

1. able to upload files and create project
2. able to view the files/projects they uploaded/created before
3. not able to see the files/projects uploaded by others
4. able to view the status of the projects/files
5. able to resend files if the original project is pending(not in process)

organization:

1. able to upload files and create project
2. able to view all the files/projects their company’s PM uploaded/created before

LLM:

1. get the files uploaded by client company’s PM
2. generate natural language file according to the files

our annotators:

1. able to view the raw files and projects assigned to them that are uploaded by organization/ client company’s PM
2. able to upload their finished annotated files
3. able to get the natural language files generated by LLM

our quality checker:

1. able to view the raw files 
2. able to view the finished annotated files 
3. able to approve/ need modification of the annotated files and able to send back to annotators
4. able to write down feedbacks

In [173]:
# --------------------------
# User Table
# --------------------------
class User(Base, TimestampMixin):
    __tablename__ = "user"
    __table_args__ = {"extend_existing": True}

    # --- Core fields ---
    user_id = Column(Integer, primary_key=True, autoincrement=True)
    email = Column(String, unique=True, nullable=False, index=True)
    #role = Column(Enum(UserRole, name="user_role_enum"), nullable=False)

    org_id = Column(
        Integer,
        ForeignKey("organization.org_id", ondelete="SET NULL"),
        nullable=True
    ) # better: keep user even if org deleted

    # --- Availability & Skills ---
    availability = Column(JSON, nullable=True)             # weekly availability
    language_expertise = Column(JSON, nullable=True)       # {"en": 4.5, "zh": 3.0}
    skill_score = Column(Float, nullable=True)             # overall skill score
    skill_level = Column(String, nullable=True)           
    qa_approval_rate = Column(Float, nullable=True)        # average QA pass rate
    completed_task_count = Column(Integer, default=0)      # total tasks completed

    # --- Relationships ---
    uploaded_files = relationship("File", back_populates="uploader")
    events = relationship("EventLog", back_populates="user")
    assignments = relationship("Assignment", back_populates="user", cascade="all, delete-orphan")

    is_active = Column(Boolean, default=True, nullable=False)
    #roles = relationship("Role", secondary=user_roles, back_populates="users")

    # PM links
    client_projects = relationship("Project", back_populates="client_pm")
    # make sure our PM has access to projects and further assign annotators
    managed_projects = relationship("Project", back_populates="our_pm")

    # Historical job links
    # Records which annotators have worked on this job before
    # (for feedback loops / reassignment tracking)
    previous_jobs = relationship(
        "AnnotationJob",
        secondary="job_previous_annotators",
        back_populates="previous_annotators"
    )

    reviews = relationship("Review", back_populates="reviewer")
    roles = relationship("Role", secondary=user_roles, back_populates="users")

/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/627162196.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  class User(Base, TimestampMixin):


    # --- Availability ---
    # Store weekly availability / working hours in JSON
    # Example:
    # {
    #   "monday": ["09:00-12:00", "13:00-17:00"],
    #   "tuesday": ["10:00-18:00"],
    #   "wednesday": []
    # }

## 3.5 Annotation Job Table

In [174]:
# --------------------------
# Annotation Job Table
# --------------------------
class AnnotationJob(Base, TimestampMixin):
    __tablename__ = "annotation_job"
    __table_args__ = {"extend_existing": True}

    job_id = Column(Integer, primary_key=True, autoincrement=True)

    # --- Foreign keys ---
    project_id = Column(Integer, ForeignKey("project.project_id", ondelete="CASCADE"), nullable=False)
    table_id = Column(Integer, ForeignKey("file_table.table_id", ondelete="CASCADE"), nullable=False)
    #file_id = Column(Integer, ForeignKey("file.file_id", ondelete="CASCADE"), nullable=True)  # optional convenience link

    # --- Job attributes ---
    language = Column(Enum(Language, name="annotation_job_language_enum"), nullable=True)
    priority = Column(Enum(JobPriority, name="job_priority_enum"), default=JobPriority.medium, nullable=False)
    status = Column(Enum(AnnotationJobStatus, name="annotation_job_status_enum"),
                    default=AnnotationJobStatus.not_started, nullable=False)
    review_status = Column(Enum(ReviewStatus, name="review_status_enum"),
                           default=ReviewStatus.pending, nullable=False)

    due_date = Column(DateTime, nullable=True)
    completed_at = Column(DateTime, nullable=True)
    is_active = Column(Boolean, default=True, nullable=False)
    deleted_at = Column(DateTime, nullable=True)

    # --- Relationships ---
    table = relationship("FileTable", back_populates="jobs")  # main link
    project = relationship("Project", back_populates="jobs")

    reviews = relationship("Review", back_populates="job", cascade="all, delete-orphan")
    assignments = relationship("Assignment", back_populates="job", cascade="all, delete-orphan")

    previous_annotators = relationship(
        "User",
        secondary="job_previous_annotators",
        back_populates="previous_jobs"
    )


/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/2184252037.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.AnnotationJob, and will be replaced in the string-lookup table.
  class AnnotationJob(Base, TimestampMixin):


## 3.6 Event Log Table

In [190]:
# --------------------------
# Event Log Table
# --------------------------
class EventLog(Base, TimestampMixin):
    __tablename__ = "event_log"
    __table_args__ = (
        Index("ix_eventlog_entity", "entity_type", "entity_id"),  # speeds up "get events for this entity"
        {"extend_existing": True},
    )

    event_id = Column(Integer, primary_key=True, autoincrement=True)
   

    # Generic entity pointer
    entity_type = Column(Enum(EntityType, name="entity_type_enum"), nullable=False)
    entity_id = Column(Integer, nullable=False)  # e.g. file_id, project_id, etc.

    # Event classification
    event_type = Column(Enum(EventType, name="event_type_enum"), nullable=False)

    # Who triggered the event
    user_id = Column(Integer, ForeignKey("user.user_id", ondelete="SET NULL"), nullable=True)
    user = relationship("User", back_populates="events")

    # Flexible metadata
    event_metadata = Column(JSONB, nullable=True)  # {"old_status": "pending", "new_status": "in_progress"}

    # Audit
    event_time = Column(DateTime, default=func.now(), nullable=False)

    # Optional direct links for efficient joins
    file_id = Column(Integer, ForeignKey("file.file_id", ondelete="SET NULL"), nullable=True)
    file_version_id = Column(Integer, ForeignKey("file_version.version_id", ondelete="SET NULL"), nullable=True)
    project_id = Column(Integer, ForeignKey("project.project_id", ondelete="SET NULL"), nullable=True)
    job_id = Column(Integer, ForeignKey("annotation_job.job_id", ondelete="SET NULL"), nullable=True)
    export_id = Column(Integer, ForeignKey("export_log.export_id", ondelete="SET NULL"), nullable=True)
    review_id = Column(
    Integer,
    ForeignKey("review.review_id", ondelete="CASCADE"),
    nullable=True
    )
    table_id = Column(Integer, ForeignKey("file_table.table_id", ondelete="SET NULL"), nullable=True)

    # Relationships
    project = relationship("Project", back_populates="events")
    file = relationship("File", back_populates="events")
    file_version = relationship("FileVersion", back_populates="events")
    job = relationship("AnnotationJob", back_populates="events")
    export = relationship("ExportLog", back_populates="events")
    review = relationship("Review", back_populates="events")
    table = relationship("FileTable", back_populates="events")

/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/4163952190.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.EventLog, and will be replaced in the string-lookup table.
  class EventLog(Base, TimestampMixin):


## 3.7 Review Table

1. quality checkers will review each annotated file done by each annotators     
2. one llm_generated file may assign to more than one annotators, so quality checker may see same file but annotated by different annotators and need to give feedback each     
3. each annotators would be able to see the reviews and clearly know the correspond file so that they can make further edition     
4. there exist condition about previous annotators, so that our PM can see who and how many people have edited one particular file     
5. each annotator can view the feedback history they gotten and know which quality checker gave which to them, each quality checker can check the feedback history they written and also to which annotator while one person can both be quality checker and annotator(but under different projects, not under same project)     
6. all reviews does not include file (just plain text wrote in the box in website) but these also needs to be stored and can not be deleted

In [176]:
# --------------------------
# Review Table
# --------------------------
class Review(Base, TimestampMixin):
    __tablename__ = "review"
    __table_args__ = (
    Index("ix_review_job_id", "job_id"),
    Index("ix_review_status", "status"),
    {"extend_existing": True},
)


    review_id = Column(Integer, primary_key=True, autoincrement=True)

    # Link to the job being reviewed
    job_id = Column(Integer, ForeignKey("annotation_job.job_id", ondelete="CASCADE"), nullable=False)

    # Reviewer (user with reviewer role)
    reviewer_id = Column(Integer, ForeignKey("user.user_id", ondelete="SET NULL"), nullable=True)

    # Review decision
    status = Column(
        Enum(ReviewStatus, name="review_table_status_enum"),
        default=ReviewStatus.pending,
        nullable=False
    )

    # Optional comments from reviewer
    feedback = Column(Text, nullable=True)


    # Audit timestamps
    # created_at = Column(DateTime, default=func.now(), nullable=False)
    # updated_at = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # Soft delete fields (prevents permanent data loss)
    is_active = Column(Boolean, default=True, nullable=False)
    deleted_at = Column(DateTime, nullable=True)


    # Relationships
    job = relationship("AnnotationJob", back_populates="reviews")
    reviewer = relationship("User", back_populates="reviews")
    events = relationship("EventLog", back_populates="review")


/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/1003076118.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Review, and will be replaced in the string-lookup table.
  class Review(Base, TimestampMixin):


## 3.8 Assignment Table

In [177]:
# --------------------------
# Assignment Table
# --------------------------
class Assignment(Base, TimestampMixin):
    __tablename__ = "assignment"
    __table_args__ = (
        Index("ix_assignment_job_id", "job_id"),
        Index("ix_assignment_user_id", "user_id"),
        Index("ix_assignment_role", "role"),
        {"extend_existing": True},
    )

    assignment_id = Column(Integer, primary_key=True, autoincrement=True)

    # Link to the annotation job (one job = one file version to be annotated)
    job_id = Column(
        Integer,
        ForeignKey("annotation_job.job_id", ondelete="CASCADE"),
        nullable=False
    )

    # Who is assigned
    user_id = Column(
        Integer,
        ForeignKey("user.user_id", ondelete="CASCADE"),
        nullable=False
    )

    # Role in this job (annotator, reviewer, qc)
    role = Column(Enum(AssignmentRole, name="assignment_role_enum"), nullable=False)

    # Status of this assignment (separate from job status)
    status = Column(
        Enum("assigned", "accepted", "in_progress", "submitted", "completed",
             name="assignment_status_enum"),
        default="assigned",
        nullable=False
    )

    # Whether this assignment is currently active (soft delete for history)
    is_active = Column(Boolean, default=True, nullable=False)
    completed_at = Column(DateTime, nullable=True)

    # Audit fields
    assigned_at = Column(DateTime, default=func.now(), nullable=False)
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # Relationships
    job = relationship("AnnotationJob", back_populates="assignments")
    user = relationship("User", back_populates="assignments")

    # Track reviews connected to this assignment (indirectly through job)
    # reviews = relationship(
    #     "Review",
    #     secondary="annotation_job",  # review links via job_id
    #     viewonly=True
    # )


/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/3084525935.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Assignment, and will be replaced in the string-lookup table.
  class Assignment(Base, TimestampMixin):


## 3.9 Role&Permission Table

1. For client company PMs, their rights and permission are:

can upload multiple files for one project and can can only view the files they themselves uploaded;

can create multiple projects and can only view the project they themselves created;

can view the annotated files finished and approved by quality checkers;

can not view the llm generated files nor view the annotated but not approved files;

allow one PM that does not belongs to any organization but not allow one same PM belongs to multiple companies;

can edit project and re-upload files as long as the project and the file are in pending(not in progress or completed…)

2. For organization, their rights and permission are:

can upload multiple files for one project and can can view the files they themselves uploaded;

can create multiple projects and can view the project they themselves created;

can view the files uploaded by their own company’s PMs, projects created by their own company’s PMs;

can view the annotated files finished and approved by quality checkers as long as these files corresponds to the files uploaded by their company’s PMs;

3. For our company product manager:

can view the projects  created by client company PMs;

can assign files to annotators;

can check the status of the projects and files;

can assign one file to multiple annotators;

can assign annotated files to quality checkers;

can reassign file to different annotators (avoiding the original annotator) (support Reassignment and Feedback Loop function);

no two PMs in our company can assign same project or files (to avoid overlaps);

can choose when to finish a project after see all the annotated files from same project completed by quality checkers;

4. For annotators:

cannot be assigned to same file more than once if it’s not from the feedback for refinement (support reassignment and Feedback Loop function);

can have multiple projects’ file annotation job;

can have feedback from quality checker and know the pointed file;

can get llm-generated file and upload their annotated files

5. For quality checker:

can make multiple feedback for same file;

can choose to complete or give feedback for each file;

(cannot have access to llm generated file, not sure about this point, whether needs to have access)

can have access to raw files uploaded by clients company PMs or organizations;

In [178]:
# --------------------------
# Role Table
# --------------------------
class Role(Base, TimestampMixin):
    __tablename__ = "role"
    __table_args__ = {"extend_existing": True}

    role_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, unique=True, nullable=False)   # e.g. "annotator", "qc", "pm"
    description = Column(Text, nullable=True)            # human-readable explanation

    # Relationships
    users = relationship("User", secondary=user_roles, back_populates="roles")
    permissions = relationship("Permission", secondary=role_permissions, back_populates="roles")

/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/3776670419.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Role, and will be replaced in the string-lookup table.
  class Role(Base, TimestampMixin):


In [179]:
# --------------------------
# Permission Table
# --------------------------
class Permission(Base, TimestampMixin):
    __tablename__ = "permission"
    __table_args__ = {"extend_existing": True}

    permission_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, unique=True, nullable=False)   # e.g. "upload_file", "assign_job", "review_annotation"
    description = Column(Text, nullable=True)            # what this permission means in plain English

    # Relationships
    roles = relationship("Role", secondary=role_permissions, back_populates="permissions")

/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/2444035327.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Permission, and will be replaced in the string-lookup table.
  class Permission(Base, TimestampMixin):


## 3.10 Organization Table

In [ ]:
from .project import Project
# ------------------------------
# Organization Table
# ------------------------------
class Organization(Base, TimestampMixin):
    __tablename__ = "organization"
    __table_args__ = (
        Index("ix_org_name", "name"),  # speed up lookups by name
        {"extend_existing": True},
    )

    org_id = Column(Integer, primary_key=True, autoincrement=True)

    # Basic info
    name = Column(String, unique=True, nullable=False)   # e.g. "Acme Corp"
    description = Column(Text, nullable=True)            # optional, for notes

    # Lifecycle & audit
    is_active = Column(Boolean, default=True, nullable=False)
    deleted_at = Column(DateTime, nullable=True)
    date_created = Column(DateTime, default=func.now(), nullable=False)
    date_updated = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # --------------------------
    # Relationships
    # --------------------------
    # All users belonging to this org
    users = relationship(
        "User",
        back_populates="organization",
        passive_deletes=True,
    )

    # All projects created under this org
    projects = relationship(
        "Project",
        back_populates="organization",
        passive_deletes=True,
    )

    # Convenience: get all files uploaded under this org (via projects)
    files = relationship(
        "File",
        secondary="project",  # indirect join via Project.project_id
        viewonly=True,
        primaryjoin="Organization.org_id==Project.org_id",
        secondaryjoin="Project.project_id==File.project_id",
    )

    # Event logs linked to this org (optional, via project/file/user actions)
    events = relationship(
        "EventLog",
        secondary=Project.__table__,
        primaryjoin="Organization.org_id == Project.org_id",
        secondaryjoin="Project.project_id == EventLog.project_id",
        viewonly=True
    )

/var/folders/c_/7thpxjgs3m38yh7ftdr5g7vm0000gn/T/ipykernel_97036/338503639.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Organization, and will be replaced in the string-lookup table.
  class Organization(Base, TimestampMixin):


## 3.11 Export / Report tables
stores generated ZIPs or audit PDFs, those should also have an S3 pointer (storage_path).

cascade behavior:    
If an export log is deleted, should the join rows in exported_file also be deleted?    
Usually yes → add cascade="all, delete-orphan" on ExportedFile if you model it as a class.    
If you keep exported_file as a raw Table, SQLAlchemy will handle cleanup when ExportLog is deleted.    


In [181]:
class ExportLog(Base, TimestampMixin):
    __tablename__ = "export_log"

    export_id = Column(Integer, primary_key=True, autoincrement=True)
    project_id = Column(Integer, ForeignKey("project.project_id"), nullable=False)
    requested_by = Column(Integer, ForeignKey("user.user_id"), nullable=False)

    # Where the final package (ZIP, PDF, TAR, etc.) lives in S3/MinIO
    storage_path = Column(String, nullable=False)

    # Optional metadata
    checksum = Column(String, nullable=True)
    #included_file_ids = Column(JSON, nullable=True)  # list of files packaged
    #included_versions = Column(JSON, nullable=True)  # if version-level tracking matters

    status = Column(Enum("pending", "completed", "failed", name="export_status_enum"), default="pending")

    date_requested = Column(DateTime, default=func.now(), nullable=False)
    date_completed = Column(DateTime, nullable=True)

    # Relationships
    project = relationship("Project", back_populates="exports")
    requested_user = relationship("User", foreign_keys=[requested_by])
    file_versions = relationship(
    "FileVersion",
    secondary="exported_file",    # uses the join table
    back_populates="exports"
    )
    exported_files = relationship(
        "ExportedFile",
        back_populates="export",
        cascade="all, delete-orphan"
    ) # Keep cascade — join table cleanup make sense here.


In [ ]:
from sqlalchemy import event, inspect

def skip_updated_at(mapper, connection, target):
    state = inspect(target)
    model_name = target.__tablename__
    real_cols = REAL_UPDATE_COLS.get(model_name, set())
    if real_cols and not any(state.attrs[c].history.has_changes() for c in real_cols):
        # none of the "real" columns changed → keep old updated_at
        target.updated_at = state.attrs["updated_at"].loaded_value

# attach to all models where needed
for model in [Project, File, FileVersion, User, AnnotationJob,
              Assignment, Review, ExportLog, Organization]:
    event.listen(model, "before_update", skip_updated_at)